### Setup

In [1]:
# gen imports
import pandas as pd, numpy as np, os, pyabc
os.chdir("C:/Users/jstelman/Git/chelsvig_urban_pesticides/src/")   # for small laptop
# os.chdir("C:/Git/chelsvig_urban_pesticides/src/")   # For Desktop
from path_names import *
from prpy_bookkeeping import *
nsims = 5

main_path   C:\Users\jstelman\Git\chelsvig_urban_pesticides
dir_path    C:\Users\jstelman\Git\chelsvig_urban_pesticides\probabilistic_python
exe_path    C:\Users\jstelman\Git\chelsvig_urban_pesticides\probabilistic_python\exe
swmm_path   C:\Users\jstelman\Git\chelsvig_urban_pesticides\probabilistic_python\input\swmm
inp_path    C:\Users\jstelman\Git\chelsvig_urban_pesticides\probabilistic_python\input\swmm\NPlesantCreek.inp
bin_path    C:\Users\jstelman\Git\chelsvig_urban_pesticides\probabilistic_python\input\swmm\NPlesantCreek.out
vvwm_path   C:\Users\jstelman\Git\chelsvig_urban_pesticides\probabilistic_python\input\vvwm
wet_path    C:\Users\jstelman\Git\chelsvig_urban_pesticides\probabilistic_python\weather


### Defining a model

(From "quickstart" example from pyabc docs)
To do model selection, we first need some models. A model, in the simplest case,
is just a callable which takes a single `dict` as input and returns a single `dict` as output. The keys of the input dictionary are the parameters of the model, the output
keys denote the summary statistics.


In [2]:
"""
Function to take in parameters (in dictionary form) and output summary statistics
 Input: 
  SWMM parameters:
    (see explanations here: <https://docs.google.com/document/d/1pVnjSP3cCKBCWUV3Brns8hJpkgHLekXns_9WG-qla7E/>)
    NImperv
    NPerv
    SImperv
    SPerv
    PctZero
    MaxRate
    MinRate
    Decay
    DryTime
    Por
    WP
    FC
    Ksat
    Roughness
    Kdecay
    BCoeff2
    WCoeff2
  
  VVWM parameters:
    (see explanations here: <https://docs.google.com/document/d/1JM8guprnugzJDvCaBwD-HCpgR5NvTGnCe3qjFXhMQHU/>)
    kd - Sorption coefficient (mL/g) -
    aer_aq - Water column degradation half-life (days) -
    aer_aq_temp - Reference temp. For water column degradation (C) -
    anae_aq - Benthic degradation half-life (days) -
    anae_aq_temp - Reference temp. For benthic degradation (C) -
    photo - Photolysis half-life (days) -
    rflat - Reference latitude for photolysis -
    hydro - Hydrolysis half-life (days) -
    mwt - Molecular weight  -
    vapor - Vapor pressure (torr) -
    sol - Solubility (mg/L) -
    henry - Henry’s law constant -
    benthic_depth - Depth of benthic region (m) -
    porosity - Porosity of benthic region (--) -
    bulk_density - Bulk density of benthic region -
    froc2 - Fraction of organic carbon on sediment in benthic region -
    doc2 - Concentration of dissolved organic carbon in benthic region (mg/L) -
    bnmas - Areal concentration of biosolids in benthic region (g/m2) -
    sused - Suspended solids concentration in water column (mg/L) -
    chl - Chlorophyll concentration in water column (mg/L) -
    froc1 - Fraction of organic carbon on suspended sediment in water column -
    doc1 - Concentration of dissolved organic carbon in water column (mg/L) -
    area - Area of water body (m2) -

 Output:
  The values of 106 date- and site- specific bifenthrin concentrations (keys are date/site code indicators)
  Why? Because that's the observation data we have access to
"""
def model(parameters: dict) -> dict:
    """
    turn swmm parameters into one csv file. 
    turn vvwm parameters into another csv file.
    if any of the the parameters are not okay:
        (that means max and min and FC and WP relationships follow rules and that everything else is in the bounds we need)
        return a dictionary full of nans and don't do the next part.
    else:
        run python scripts 03-06 and 08-09 (i/Ite = c)
        run some derivation of postprocess notebook to take 106 cells of that file made by 09 and turn it into a dictionary
    return (the dictionary)
    """
    pass

### File 02 would be here, but actually, it will take place in the prior segment

In [2]:
"""
Simulate dict of LHS parameters from the first row of the dfs of swmm parameters and of vvwm parameters
"""
lhs_design = pd.concat([pd.read_csv(os.path.join(dir_path, "io", "lhs_sampled_params.csv"), index_col=0),
                        pd.read_csv(os.path.join(dir_path, "io", "lhs_sampled_params_vvwm.csv"), index_col=0)],
                      axis = 1)
# round lhs decimals
lhs_design = lhs_design.round(
    {   # swmm
        "NImperv": 4, "NPerv": 4, "SImperv":3, "SPerv": 3, "PctZero": 3, "MaxRate": 3, "MinRate": 3, "Decay": 2, 
        "DryTime": 0, "Por": 3, "WP": 3, "FC": 3, "Ksat": 3, "Rough": 4, "Kdecay": 3, "BCoeff2": 3, "WCoeff2": 3,
        # vvwm
        "kd": 2, "aer_aq": 0, "aer_aq_temp": 2, "anae_aq": 0, "anae_aq_temp": 2, "photo": 2, "rflat": 0, "hydro": 2, 
        "sol": 4, "benthic_depth": 3, "porosity": 2, "bulk_density": 2, "froc2": 3, "doc2": 2, "bnmas": 3, "sused": 3, 
        "chl": 3, "froc1": 3, "doc1": 2
    })
# SWMM lhs parameter (1 row) as dictionary
params1 = lhs_design.to_dict(orient = 'records')[0]   

In [3]:
params1

{'NImperv': 0.0179,
 'NPerv': 0.3951,
 'SImperv': 1.363,
 'SPerv': 4.442,
 'PctZero': 15.2,
 'MaxRate': 81.273,
 'MinRate': 7.266,
 'Decay': 6.26,
 'DryTime': 8.0,
 'Por': 0.465,
 'FC': 0.361,
 'WP': 0.119,
 'Ksat': 1.719,
 'Rough': 0.0194,
 'Kdecay': 0.045,
 'BCoeff2': 1.584,
 'WCoeff2': 0.203,
 'kd': 5333.21,
 'aer_aq': 272.0,
 'aer_aq_temp': 22.96,
 'anae_aq': 255.0,
 'anae_aq_temp': 23.76,
 'photo': 404.98,
 'rflat': 40.0,
 'hydro': 106.3,
 'sol': 0.48,
 'benthic_depth': 0.884,
 'porosity': 0.71,
 'bulk_density': 1.61,
 'froc2': 0.02,
 'doc2': 27.62,
 'bnmas': 1.094,
 'sused': 51.269,
 'chl': 0.984,
 'froc1': 0.113,
 'doc1': 0.64}

### Begin function guts

In [4]:
'''
Then we would have to split them into two after they have been passed into the function
'''
swmm_keys = list(params1.keys())[:17]
vvwm_keys = list(params1.keys())[17:]
swmm_params = {key: params1[key] for key in swmm_keys}
vvwm_params = {key: params1[key] for key in vvwm_keys}

### From 03

In [6]:
import pytest_shutil, shutil, regex as re, dask, uuid
loginfo = log_prefixer("03")

In [7]:
# Helper Functions

'''
Helper function of edit_file:
Create new line of .inp file with lhs simulated versions of values and text in old .inp file
 Inputs: fileline <str> -line from original .inp file-
   Col <int> -column in the .inp file that contains the info you want to preserve and clean- 
   sim <float> -the simulated value to repace the old (observed) one with-
 Output: newline <str> -custom version of line for new file-
'''
def edit1line(fileline, Col, sim):
    listline = fileline.split()
    listline[Col] = str(sim)
    newline = ' '.join([str(item) for item in listline]) + "\n"
    return(newline)

# From 03 with edits

'''
Edit the new file to be cleaner
 Inputs: Ite <int> -index of current simulation-
   Num <int> -Number of rows to clean-
   row_0 <int> -Number of rows to skip-
   parameter <str> -name of lhs design parameter (will become name of column in the .csv  file)-
   Col <int> -column in the .inp file that contains the info you want to preserve and clean- 
   flines <list of str> -lines of the file to clean up-
 Output: cleaned up lines of file given
'''
def editted_lines(swmm_dict, Num, row_0, parameter, Col, flines):
    # value of "parameter" in current lhs simulation
    # sim = lhs_design.loc[Ite - 1, parameter]   # updating
    sim = swmm_dict[parameter]
    print(sim)
    return([edit1line(flines[row_0 + i], Col, sim) for i in range(Num)])


In [11]:
'''
To eliminate the need to open .inp file in 05, let's create a variable with the info we need.
Afterall, it never changes
'''

# create blank list to hold subcatchment areas
sub_list_area = []

with open(inp_path, "r") as ip_file:
    lines1 = ip_file.readlines()[55:]
    # [sub_list_area.append(float(lines1[thissub].split()[3])) for thissub in range(113)]   ## Too smooshed
    for thissub in range(113):
        # grab the area
        listline = lines1[thissub].split()
        area = float(listline[3])
        # insert into blank list
        sub_list_area.append(area)

[57.8,
 40.6289,
 227.649,
 10.2923,
 158.7419,
 30.2337,
 53.5458,
 43.0476,
 56.2647,
 72.3722,
 39.4796,
 31.7947,
 87.905,
 280.1313,
 40.526,
 39.5054,
 5.5064,
 60.9305,
 25.9281,
 92.8453,
 104.4928,
 53.7259,
 23.6466,
 44.3771,
 23.1835,
 56.0074,
 46.8043,
 135.3097,
 154.5049,
 35.8344,
 26.8544,
 48.1595,
 57.5169,
 51.7446,
 50.3809,
 6.6471,
 66.2397,
 74.9281,
 8.4997,
 36.0574,
 21.0993,
 79.0193,
 69.2587,
 80.0571,
 2.6503,
 24.0068,
 4.9489,
 74.4821,
 90.2294,
 113.4728,
 41.9241,
 94.312,
 25.748,
 40.3802,
 108.524,
 79.8684,
 147.995,
 75.803,
 6.8701,
 127.1102,
 19.5039,
 4.4085,
 60.8105,
 68.6669,
 0.8234,
 127.5304,
 111.2343,
 147.8492,
 25.7394,
 7.6849,
 53.563,
 125.2147,
 43.7166,
 49.0343,
 24.0411,
 33.2442,
 514.3245,
 134.0832,
 58.8378,
 88.2481,
 2.9848,
 43.0648,
 140.7132,
 4.5801,
 106.5427,
 42.6016,
 264.6327,
 60.7247,
 75.6829,
 24.2899,
 163.5621,
 26.2883,
 15.8159,
 17.7199,
 40.0629,
 15.1126,
 34.2992,
 61.3251,
 40.6289,
 41.8812,
 29

# '''
let's make a file to hold the copied input files
'''

In [ ]:
'''
Step 1:
Generate random id (see "uuid.uuid4().hex[0:8]     #generate random jobID" in Jeff's code on bookmarks bar)
Make input and output file names incorporate this id (so things don't get lost or overwritten thanks to parallelism. Yay)
Also make DLL copy incorporate this id 

Step 2:
Run the thing!
Run swmm toolbox on it
Then delete that massive output file and the dll.
Honestly, maybe even the input file. 

(Step 3:
Carry on with the rest of the munging til the VVWM step. 

Step 4:
Do something similar to step 1 for VVWM)
'''

In [16]:
"""
Simulate the first step of running the model for one iteration
"""

# generate random id
Ite = i = rpt = uuid.uuid4().hex[0:8]

In [17]:
# make the folder, WE SHOULD TRY TO SKIP THIS
# MAKE JUST ONE DIR FOR ALL CREATED inp FILES, ONE FOR ALL CREATED out FILES, AND ONE FOR ALL CREATED dll FILES
new_dir = os.path.join(swmm_path, "input_" + Ite)

if not os.path.exists(new_dir):
    os.mkdir(new_dir)
    print("Folder ", Ite, " created", "\n")
else:
    print("Folder ", Ite, "already exists")

Folder  09cd7b5f  created 



In [18]:
# # copy base file into new file location
#### THIS SHIT HAS ALREADY BEEN NAMED "inp_path" IN PATH_NAMES
# old_path = os.path.join(swmm_path, "NPlesantCreek.inp")
# old_path = inp_path
# RIGHT HERE!!!!!!!! temp it instead.... but what to do with the output?
new_path = os.path.join(new_dir, "NPlesantCreek.inp")
# loginfo("Copying base swmm input file <" + old_path + "> into <" + new_dir + ">.")
# shutil.copyfile(old_path, new_path)
# shutil.copyfile(inp_path, new_path)

In [34]:
with open(inp_path, "r") as read_file, open(new_path, "w") as write_file:
    filelines = read_file.readlines()

    # first we need to correct some absolute paths, because they are currently only set to work on the author's computer
    filelines = replace_infile_abspaths(filelines = filelines)
    
    # 113 = number of subcatchments
    for c, par in enumerate(["NImperv", "NPerv", "SImperv", "SPerv", "PctZero"]):
        filelines[172:(172 + 113)] = editted_lines(swmm_dict = swmm_params, Num = 113, row_0 = 172, parameter = par, Col = c+1, flines = filelines)
    for c, par in enumerate(["MaxRate", "MinRate", "Decay", "DryTime"]):
        filelines[289:(289 + 113)] = editted_lines(swmm_dict = swmm_params, Num = 113, row_0 = 289, parameter = par, Col = c+1, flines = filelines)

    # 1 = number of aquifers
    for c, par in enumerate(["Por", "WP", "FC", "Ksat"]):
        filelines[406:(406 + 1)] = editted_lines(swmm_dict = swmm_params, Num = 1, row_0 = 406, parameter = par, Col = c+1, flines = filelines)

    # 195 = number of conduits
    # filelines[734:(734 + 195)] = editted_lines(swmm_dict = swmm_params, Num = 195, row_0 = 734, parameter = "Rough", Col = 4, flines = filelines)

    # 1 = number of pollutants
    filelines[1125:(1125 + 1)] = editted_lines(swmm_dict = swmm_params, Num = 1, row_0 = 1125, parameter = "Kdecay", Col = 5, flines = filelines)
    filelines[1371:(1371 + 1)] = editted_lines(swmm_dict = swmm_params, Num = 1, row_0 = 1371, parameter = "BCoeff2", Col = 4, flines = filelines)
    filelines[1377:(1377 + 1)] = editted_lines(swmm_dict = swmm_params, Num = 1, row_0 = 1377, parameter = "WCoeff2", Col = 4, flines = filelines)
    
    write_file.writelines(filelines)


0.0179
0.3951
1.363
4.442
15.2
81.273
7.266
6.26
8.0
0.465
0.119
0.361
1.719
0.045
1.584
0.203


### From 04

In [27]:
from pyswmm import Simulation
import swmmtoolbox.swmmtoolbox as swmmtoolbox
import time
from pyswmm.lib import DLL_SELECTION
loginfo = log_prefixer("04")
# WE ALREADY HAVE new_dir
# inp_dir_prefix = os.path.join(dir_path, "input", "swmm", "input_")

In [28]:
# Make DLL folder

# save the path for the original dynamic link library
dll_path = DLL_SELECTION()
# save its base name (just the name of the file)
dll_bn = os.path.basename(dll_path)
# save the path to a new folder where copies of dll will be stored upon creation
dll_dir = os.path.join(dir_path, "input", "swmm", "dll")
# make sure that folder exists. If it doesn't, create it.
if not os.path.exists(dll_dir):
    loginfo("Creating directory <" + dll_dir + ">.")
    print("Creating <dll/> directory.")
    os.mkdir(dll_dir)

In [36]:
# log something generic
# loginfo("Simmulation " + str(i) + " of " + str(nsims))

# create path to the 1-time-use dll file we are going to create
dll_i = dll_bn[:dll_bn.rindex(".")] + '-' + i + dll_bn[dll_bn.rindex("."):]
lib_path = os.path.join(dll_dir, dll_i)
# create 1-time-use dll file copy
shutil.copyfile(dll_path, lib_path)

'C:\\Users\\jstelman\\Git\\chelsvig_urban_pesticides\\probabilistic_python\\input\\swmm\\dll\\swmm5-x64-09cd7b5f.dll'

In [40]:
# specify the directory with the file pyswmm needs by attaching the folder id to the rest of the folder's absolute path
# sim_dir = inp_dir_prefix + str(i)
# specify the actual file pyswmm needs
# WE ALREADY HAVE THIS, IT'S CALLED "new_path"
# sim_path = os.path.join(sim_dir, r'NPlesantCreek.inp')
#print("Simulation input file found:", sim_path)

# specify the file that pyswmm will (over)write with output after running the probabilistic simulation
# THIS IS NOT REQUIRED YET, BUT WHY NOT BE EXPLICIT
sim_bin_path = os.path.join(new_dir, "NPlesantCreek.out") #sim_dir, "NPlesantCreek.out")
# delete pre-existing .out, if present, in order to run swmm agreeably
if os.path.exists(sim_bin_path):
    loginfo("Deleting current copy of <" + sim_bin_path + "> so new copy can be created.")
    #print("Deleting current copy of <NPlesantCreek.out> so new copy can be created.")
    os.remove(sim_bin_path)

# stagger starting times 1 sec apart
# time.sleep(i)

# load the model {no interaction, write (binary) results to <JS_NPlesantCreek.out>, use the specified dll}
sim = Simulation(inputfile=new_path, reportfile=None, outputfile=sim_bin_path, swmm_lib_path=lib_path)
# simulate the loaded model
loginfo("Executing SWMM simmulation with no interaction. Input from <" + new_path + ">. Will store output in <" + sim_bin_path + ">.")
# sim.execute()
with sim as s:
    for step in s:
        pass

# extract swmm outputs with swmmtoolbox and delete expensive binary files
lab1 = 'subcatchment,,Runoff_rate'
lab2 = 'subcatchment,,Bifenthrin'
runf_stack = swmmtoolbox.extract(sim_bin_path, lab1)
bif_stack = swmmtoolbox.extract(sim_bin_path, lab2)

loginfo("Deleting <" + sim_bin_path + "> to free up memory.")
os.remove(sim_bin_path)
loginfo("Deleting <" + dll_i + "> to free up memory.")
os.system("rm " + lib_path)
print("Deleted <input_" + str(i) + "/JS_NPlesantCreek.out> and <" + dll_i + "> to free up memory.")

loginfo("Deleting <" + sim_bin_path[:-3]+"rpt" + "> to free up memory.")
os.remove(sim_bin_path[:-3]+"rpt")
loginfo("Deleting <" + new_path + "> to free up memory.")
os.remove(new_path)
# print("Deleted <input_" + str(i) + "/NPlesantCreek.out> and <" + dll_i + "> to free up memory.")

# compute and export daily averages to csv files and finish
runf_davg = runf_stack.resample('D').mean()
bif_davg = bif_stack.resample('D').mean()
# print(save_and_finish(runf_davg, os.path.join(sim_dir, "swmm_output_davg_runf.csv")))
# print(save_and_finish(bif_davg, os.path.join(sim_dir, "swmm_output_davg_bif.csv")))
# msg1 and msg2 text will be the same, but we must do both to save both csvs

INFO:root:04/02/2021 08:45:26:: 04: Executing SWMM simmulation with no interaction. Input from <C:\Users\jstelman\Git\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_09cd7b5f\NPlesantCreek.inp>. Will store output in <C:\Users\jstelman\Git\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_09cd7b5f\NPlesantCreek.out>.
INFO:root:04/02/2021 09:06:55:: 04: Deleting <C:\Users\jstelman\Git\chelsvig_urban_pesticides\probabilistic_python\input\swmm\input_09cd7b5f\NPlesantCreek.out> to free up memory.
INFO:root:04/02/2021 09:06:56:: 04: Deleting <swmm5-x64-09cd7b5f.dll> to free up memory.


Deleted <input_09cd7b5f/JS_NPlesantCreek.out> and <swmm5-x64-09cd7b5f.dll> to free up memory.


### From 05

In [49]:
'''
Make this slicer object once, for all.
Use slicer[o] to reference the thing you want to reference
Honestly, in the future, we will probably change it to a dot product anyway. And replace the NaNs with 0s
'''
outfalls = ['outfall_31_26', 'outfall_31_28', 'outfall_31_29', 'outfall_31_35',
            'outfall_31_36', 'outfall_31_38', 'outfall_31_42']
slicer = {}
for o in outfalls:
    outfall_dir = os.path.join(vvwm_path, o)
    # subset subcatchment outputs for each vvwm
    outfall_path = os.path.join(outfall_dir, o + ".csv")
    # declare which columns need to be subset
    sub_ids = (pd.read_csv(outfall_path, header = 0, usecols=['Subcatchment_ID']) - 1).sum(1).tolist()

    # Now, to subset:
    # make list where the elements at included indices are 1s and the other elements are 0s
    slicer[o] = [(1 if x in sub_ids else np.nan) for x in range(113)]

{'outfall_31_26': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  nan,
  nan,
  nan,
  1,
  nan,
  1,
  nan,
  nan,
  1,
  1,
  nan,
  1,
  nan,
  nan,
  1,
  1,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  1,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 'outfall_31_28': [nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  1,


In [51]:
# inp_dir_prefix = os.path.join(dir_path, "input", "swmm", "input_")

# # save the date columns for the csvs we will be making at the very end. 
# datedf = pd.DataFrame({"date": pd.date_range(start='1/1/2009', periods=3287, freq='D')})
# datedf['year'] = datedf['date'].dt.year
# datedf['month'] = datedf['date'].dt.month
# datedf['day'] = datedf['date'].dt.day
# dates = datedf.date.tolist()
# years = datedf.year.tolist()
# months = datedf.month.tolist()
# days = datedf.day.tolist()

# runf_df_cols, runf_df_rows, bif_df_cols, bif_df_rows = 113, 3287, 113, 3287



In [58]:
pd.DatetimeIndex(pd.date_range(start='1/1/2009', periods=3287, freq='D')) == runf_davg.index

array([ True,  True,  True, ...,  True,  True,  True])

In [ ]:
for o in outfalls:
    # set pathways
    outfall_dir = os.path.join(vvwm_path, o)

    # create vvwm prob. sim. input folders
    for Ite in range(1, nsims + 1):
        new_dir = os.path.join(outfall_dir, "input_" + str(Ite))

        if not os.path.exists(new_dir):
            os.mkdir(new_dir)
            print("Folder ", Ite, " created")
        else:
            print("Folder ", Ite, "already exists")


In [ ]:
    # write out swmm daily average outputs
#     runf_to_conv = dask.delayed(pd.read_csv)(os.path.join(sim_dir, "swmm_output_davg_runf.csv"), index_col = 0)
#     bif_to_conv = dask.delayed(pd.read_csv)(os.path.join(sim_dir, "swmm_output_davg_bif.csv"), index_col = 0)
    runf_to_conv = runf_davg
    bif_to_conv = bif_davg

    # Conversion for runf and bif
    runf_to_conv = runf_to_conv.mul(86400).mul(0.01).div(sub_list_area)
    bif_to_conv = bif_to_conv.mul(runf_to_conv.values)

    # Write out converted swmm outputs for runoff and bifenthrin
#     runf_to_conv = dask.delayed(save_and_continue)(runf_to_conv, os.path.join(sim_dir, "swmm_conv_to_vvwm_runf.csv"))
#     bif_to_conv = dask.delayed(save_and_continue)(bif_to_conv, os.path.join(sim_dir, "swmm_conv_to_vvwm_bif.csv"))

    for o in outfalls:
        outfall_dir = os.path.join(vvwm_path, o)

In [ ]:
#         # subset subcatchment outputs for each vvwm
#         outfall_path = os.path.join(outfall_dir, o + ".csv")
#         # declare which columns need to be subset
#         sub_ids = (pd.read_csv(outfall_path, header = 0, usecols=['Subcatchment_ID']) - 1).sum(1).tolist()
        
#         # Now, to subset:
#         # make list where the elements at included indices are 1s and the other elements are 0s
#         slicer = [(1 if x in sub_ids else np.nan) for x in range(113)]
        # now multiply it by the df
        runf_sub = (runf_to_conv * slicer[o]).dropna(1)
        bif_sub = (bif_to_conv * slicer[o]).dropna(1)

In [ ]:
        # add a total sum column and date columns
        runf_sub=runf_sub.assign(runf_sum = runf_sub.sum(axis=1), date = dates, year = years, month = months, day = days)
        bif_sub=bif_sub.assign(bif_sum = bif_sub.sum(axis=1), date = dates, year = years, month = months, day = days)

#         # write out dataframes
#         sfx_o = outfall_path[-9:]
#         runf_out = os.path.join(outfall_dir, "input_" + str(rpt), "runf_for_vvwm" + sfx_o)
#         bif_out = os.path.join(outfall_dir, "input_" + str(rpt), "bif_for_vvwm" + sfx_o)

### From 06

In [ ]:
'''
Now, we are essentially going to only keep the total and date columns. 
(So a lot of 05 might be editted to avoid wasted effort.)


'''

### Now we need priors 

Not sure how to do that because there seems to be a much stricter method for prior declaration. 
Maybe I can do this first step outside
then enter it as a a normal distribution with a sigma of 0 for all the parameters that are special

In [79]:
"""
Priors. Get the values from that csv. Just for SWMM at first.
"""
swmm_ranges = pd.read_csv(os.path.join(dir_path, "input", "lhs", "lhs_param_ranges.csv"), index_col=0,
                           usecols = ["Parameter","Min", "Range"])

In [80]:
'''
Link up with the vvwm priors and make one big list with 36 params
'''
vvwm_ranges = pd.read_csv(os.path.join(dir_path, "input", "lhs", "lhs_param_ranges_vvwm.csv"), index_col=0,
                           usecols = ["Parameter","Min", "Range"])

param_ranges = pd.concat([swmm_ranges, vvwm_ranges], axis = 0)

priors = param_ranges.to_dict("index")

# borrowed from Jeff: <https://github.com/JeffreyMinucci/bee_neonic_abc/blob/master/pyabc_run.ipynb>
prior = pyabc.Distribution(**{key: pyabc.RV("uniform", loc = v['Min'], scale = v['Range'])
                        for key, v in priors.items()})
prior


<Distribution 'BCoeff2', 'Decay', 'DryTime', 'FC', 'Kdecay', 'Ksat', 'MaxRate', 'MinRate', 'NImperv', 'NPerv', 'PctZero', 'Por', 'Rough', 'SImperv', 'SPerv', 'WCoeff2', 'WP', 'aer_aq', 'aer_aq_temp', 'anae_aq', 'anae_aq_temp', 'benthic_depth', 'bnmas', 'bulk_density', 'chl', 'doc1', 'doc2', 'froc1', 'froc2', 'hydro', 'kd', 'photo', 'porosity', 'rflat', 'sol', 'sused'>

In [77]:
param_ranges

,Min,Range
Parameter,,
NImperv,0.010000,0.015000
NPerv,0.050000,0.450000
SImperv,1.270000,1.270000
SPerv,2.540000,2.540000
PctZero,0.010000,99.990000
MaxRate,8.460000,118.540000
MinRate,0.254000,10.666000
Decay,2.000000,5.000000
DryTime,2.000000,12.000000


### So then I could do something like this

In [ ]:
abc = ABCSMC(model, prior, distance = hydroeval.NSE)

### Then I need to set the observed data 

I will have to make that csv into a dictionary like how Jeff did it. 
This should be the easiest part
I've basically already done some of it in the post-processing notebook.